In [ ]:
import quandl
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import numpy as np
quandl.ApiConfig.api_key = 's_YHBWCvr5YJr3-2ifzU' # key 

In [ ]:
data = quandl.get('EURONEXT/', start_date='2018-01-02', end_date='2018-12-04')

In [ ]:
data.Close.plot()
plt.show()

In [ ]:
data_analysis = data

# basic analysis

In [ ]:
data_analysis['100ma'] = data_analysis['Close'].rolling(window=100,min_periods=0).mean()
# this is used to compute the average for pass 100 price (including itself)

ax1 = plt.subplot2grid((6,1),(0,0),rowspan=5,colspan=1)
ax2 = plt.subplot2grid((6,1),(5,0),rowspan=5,colspan=1,sharex=ax1)
ax1.plot(data.index,data_analysis['Close'])
ax1.plot(data.index,data_analysis['100ma'])
ax2.plot(data.index,data_analysis['Total Trade Quantity'])
plt.show()

In [ ]:
data_ohlc = data_analysis['Close'].resample('10D').ohlc()
data_volume = data_analysis['Total Trade Quantity'].resample('10D').sum()
data_ohlc.reset_index(inplace=True) # the index should be 0 1 2 ....
data_ohlc['Date'] = data_ohlc['Date'].map(mdates.date2num) # transfer datetime to timestamp

ax1 = plt.subplot2grid((6,1),(0,0),rowspan=5,colspan=1)
ax2 = plt.subplot2grid((6,1),(5,0),rowspan=5,colspan=1,sharex=ax1)
ax1.xaxis_date()
candlestick_ohlc(ax1,data_ohlc.values,width=2,colorup='g')
ax2.fill_between(data_volume.index.map(mdates.date2num),data_volume.values,0)
plt.show() ## candlestick graph

## calculation of Relative Strength index(RSI)

In [ ]:
TechIndicator = data.copy()

In [ ]:
# Relative Strength Index
# Avg(PriceUp)/(Avg(PriceUP)+Avg(PriceDown)*100
# Where: PriceUp(t)=1*(Price(t)-Price(t-1)){Price(t)- Price(t-1)>0};
#        PriceDown(t)=-1*(Price(t)-Price(t-1)){Price(t)- Price(t-1)<0};

def rsi(values):
    up = values[values>0].mean()
    down = -1*values[values<0].mean()
    return 100 * up / (up + down)

In [ ]:
# Add Momentum_1D column for all 15 stocks.
# Momentum_1D = P(t) - P(t-1)
TechIndicator['Momentum_1D'] = (TechIndicator['Close']-TechIndicator['Close'].shift(1)).fillna(0)
TechIndicator['RSI_14D'] = TechIndicator['Momentum_1D'].rolling(center=False, window=14).apply(rsi).fillna(0)
TechIndicator.tail(5)

## Calculation of Bollinger Bands

In [ ]:
def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    #ave = pd.stats.moments.rolling_mean(price,length)
    ave = price.rolling(window = length, center = False).mean()
    #sd = pd.stats.moments.rolling_std(price,length)
    sd = price.rolling(window = length, center = False).std()
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

In [ ]:
TechIndicator['BB_Middle_Band'], TechIndicator['BB_Upper_Band'], TechIndicator['BB_Lower_Band'] = bbands(TechIndicator['Close'], length=20, numsd=1)
TechIndicator['BB_Middle_Band'] = TechIndicator['BB_Middle_Band'].fillna(0)
TechIndicator['BB_Upper_Band'] = TechIndicator['BB_Upper_Band'].fillna(0)
TechIndicator['BB_Lower_Band'] = TechIndicator['BB_Lower_Band'].fillna(0)
TechIndicator.tail()

## Calculation of Aroon Oscillator

In [ ]:
def aroon(df, tf=25):
    aroonup = []
    aroondown = []
    x = tf
    while x< len(df['High']):
        aroon_up = ((df['High'][x-tf:x].tolist().index(max(df['High'][x-tf:x])))/float(tf))*100
        aroon_down = ((df['Low'][x-tf:x].tolist().index(min(df['Low'][x-tf:x])))/float(tf))*100
        aroonup.append(aroon_up)
        aroondown.append(aroon_down)
        x+=1
    return aroonup, aroondown

In [ ]:
listofzeros = [0] * 25
up, down = aroon(TechIndicator)
aroon_list = [x - y for x, y in zip(up,down)]
if len(aroon_list)==0:
    aroon_list = [0] * TechIndicator.shape[0]
    TechIndicator['Aroon_Oscillator'] = aroon_list
else:
    TechIndicator['Aroon_Oscillator'] = listofzeros+aroon_list

## Calculation of Price Volume Trend
PVT = [((CurrentClose - PreviousClose) / PreviousClose) x Volume] + PreviousPVT

In [ ]:
TechIndicator["PVT"] = (TechIndicator['Momentum_1D']/ TechIndicator['Close'].shift(1))*TechIndicator['Total Trade Quantity']
TechIndicator["PVT"] = TechIndicator["PVT"]-TechIndicator["PVT"].shift(1)
TechIndicator["PVT"] = TechIndicator["PVT"].fillna(0)

## Calculation of Acceleration Bands

# Input data from yahoo 

In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()


Since we don't have the tickers names of companies in LSE(London Stock Exchange), we need to input all tickers names at first 

In [2]:
data = pd.read_csv('LSE.txt')

In [3]:
def splitTicker(row):
    splitname = row['Symbol\tDescription'].split('\t')
    return splitname[0]
def splitCompany(row):
    splitname = row['Symbol\tDescription'].split('\t')
    return splitname[1]
def lowerName(row):
    new_name =  row['company_name'].lower()
    return new_name
def transferDatetime(row):
    time = datetime.strptime(str(row['addmission_time']), '%d/%m/%Y')
    return time
def filterAddmissionTime(data,past_time):
    return data[data['addmission_time']<=past_time]

In [4]:
data['ticker']= data.apply(splitTicker,axis=1)
data['company_name'] = data.apply(splitCompany,axis=1)
data['company_name'] = data.apply(lowerName,axis=1)
data['company_name'].head()

0    statoilhydro asa 6.125% notes 27/11/28 gbp(var)
1                                 baywa ag npv(vink)
2                       crcam nord de france eur3.05
3                                 carlsberg `a`dkk20
4                                 carlsberg `b`dkk20
Name: company_name, dtype: object

In [5]:
company = pd.read_csv('company.csv')
company = company[['Unnamed: 1','Unnamed: 2']][5:-2]
company.columns=['addmission_time','company_name']
company['company_name'] = company.apply(lowerName,axis=1)
company.dropna(inplace=True)
company['addmission_time'] = company.apply(transferDatetime,axis=1)

In [6]:
cleaned_data = company.merge(data,on=['company_name']).drop(columns='Symbol\tDescription')
cleaned_data.head()

,addmission_time,company_name,ticker
0,2006-08-02 00:00:00,1pm plc,OPM.L
1,2009-02-02 00:00:00,1spatial plc,SPA.L
2,1994-07-18 00:00:00,3i group plc,III.L
3,2014-02-18 00:00:00,4d pharma plc,DDDD.L
4,1953-03-13 00:00:00,4imprint group plc,FOUR.L


In [7]:
past_time = datetime.now()-timedelta(days=3650)

filtered_date = filterAddmissionTime(cleaned_data,past_time)

In [8]:
tickers_list =  list(filtered_date['ticker'])

In [13]:
Total_data = pdr.get_data_yahoo(tickers=tickers_list,start=past_time,end=datetime.now())

[*********************100%***********************]  1025 of 1025 downloaded


In [53]:
Total_data.head()

Open                                                           \
           AADV.L AAEV.L AAIF.L   AAL.L   AAM.L  AAS.L AATG.L AAU.L AAVC.L   
Date                                                                         
2009-01-12  0.735    0.7  1.020  1474.0  0.5525  2.330    0.7  1.75  0.625   
2009-01-13  0.735    0.7  1.000  1422.0  0.5525  2.320    0.7  1.75  0.600   
2009-01-14  0.735    0.7  1.010  1454.0  0.5525  2.340    0.7  1.75  0.625   
2009-01-15  0.735    0.7  0.990  1289.0  0.5525  2.335    0.7  1.75  0.625   
2009-01-16  0.735    0.7  1.025  1312.0  0.5525  2.310    0.7  1.75  0.600   

                   ...     Volume                                              \
           AAZ.L   ...      WYN.L      XAR.L     XPP.L YGEN.L     YOU.L ZIN.L   
Date               ...                                                          
2009-01-12   6.0   ...        0.0   140000.0   37200.0    NaN  219886.0   0.0   
2009-01-13   6.0   ...        0.0   104815.0    4940.0    NaN  146674.0   0.0   
2009-01-14   7.0   ...        0.0    31485.0    4083.0    NaN  352736.0   0.0   
2009-01-15   7.0   ...     4500.0  1472514.0  582562.0    NaN  188571.0   0.0   
2009-01-16   7.0   ...        0.0   237837.0       0.0    NaN  140219.0   0.0   

                                               
              ZOL.L    ZOO.L   ZTF.L    ZYT.L  
Date                                           
2009-01-12      0.0  24400.0  7255.0  12763.0  
2009-01-13      0.0   1363.0     0.0   3500.0  
2009-01-14      0.0      0.0     0.0      0.0  
2009-01-15   2000.0      0.0     0.0      0.0  
2009-01-16  35000.0   3279.0     0.0      0.0  

[5 rows x 2526 columns]

## seperate the dataframes into several dataframes by closing price and total price 
we can get volume value and closing price 

In [60]:
Closed_price = Total_data['Close']
missing_percent = Closed_price.isna().sum()/len(Closed_price)
cleaned_cloing_price = Closed_price.drop(columns=missing_percent[missing_percent>0.01].index)
cleaned_cloing_price.fillna(method='ffill',inplace=True)
cleaned_cloing_price.head()

,AADV.L,AAEV.L,AAIF.L,AAL.L,AAM.L,AAS.L,AATG.L,AAU.L,AAVC.L,AAZ.L,...,WWH.L,WYN.L,XAR.L,XPP.L,YOU.L,ZIN.L,ZOL.L,ZOO.L,ZTF.L,ZYT.L
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,0.735,0.7,1.015,1451.0,0.5525,2.3575,0.7,1.75,0.625,5.88,...,5.40,172.5,48.0,131.00,85.00,265.0,31.0,8.810,59.5,110.5
2009-01-13,0.735,0.7,0.995,1429.0,0.5525,2.3525,0.7,1.75,0.625,6.00,...,5.47,172.5,49.0,129.00,84.00,265.0,31.0,9.130,59.5,110.5
2009-01-14,0.735,0.7,1.010,1293.0,0.5525,2.4000,0.7,1.75,0.625,7.00,...,5.50,172.5,47.5,129.25,77.50,265.0,31.0,9.125,59.5,110.5
2009-01-15,0.735,0.7,1.005,1275.0,0.5525,2.3350,0.7,1.75,0.625,7.00,...,5.38,167.0,48.0,129.50,70.75,260.0,17.0,9.125,59.5,106.0
2009-01-16,0.735,0.7,0.985,1371.0,0.5525,2.3100,0.7,1.88,0.625,6.50,...,5.35,167.0,47.0,129.25,72.00,260.0,13.2,9.130,59.5,106.0


In [63]:
columns_list = cleaned_cloing_price.columns
Volume = Total_data['Volume']
cleaned_volume = Volume[columns_list]

In [64]:
cleaned_volume

,AADV.L,AAEV.L,AAIF.L,AAL.L,AAM.L,AAS.L,AATG.L,AAU.L,AAVC.L,AAZ.L,...,WWH.L,WYN.L,XAR.L,XPP.L,YOU.L,ZIN.L,ZOL.L,ZOO.L,ZTF.L,ZYT.L
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,0.0,0.0,111937.0,5593445.0,0.0,25439.0,0.0,0.0,0.0,170313.0,...,28700.0,0.0,140000.0,37200.0,219886.0,0.0,0.0,24400.0,7255.0,12763.0
2009-01-13,0.0,0.0,51502.0,8228871.0,0.0,20734.0,0.0,0.0,3000.0,1179779.0,...,53085.0,0.0,104815.0,4940.0,146674.0,0.0,0.0,1363.0,0.0,3500.0
2009-01-14,0.0,0.0,79242.0,9075464.0,0.0,30045.0,0.0,0.0,0.0,0.0,...,63186.0,0.0,31485.0,4083.0,352736.0,0.0,0.0,0.0,0.0,0.0
2009-01-15,0.0,0.0,154148.0,11590893.0,0.0,10036.0,0.0,0.0,0.0,52045.0,...,22648.0,4500.0,1472514.0,582562.0,188571.0,0.0,2000.0,0.0,0.0,0.0
2009-01-16,0.0,0.0,25903.0,9478963.0,0.0,13254.0,0.0,0.0,10218.0,302291.0,...,28029.0,0.0,237837.0,0.0,140219.0,0.0,35000.0,3279.0,0.0,0.0
2009-01-19,0.0,0.0,68950.0,6619279.0,0.0,20200.0,0.0,1964000.0,0.0,275291.0,...,22960.0,4367.0,42754.0,28937.0,35801.0,2300.0,0.0,351.0,0.0,0.0
2009-01-20,0.0,0.0,52881.0,8138538.0,0.0,35664.0,0.0,860000.0,0.0,180689.0,...,24571.0,0.0,181088.0,17924.0,19319.0,0.0,0.0,0.0,0.0,0.0
2009-01-21,0.0,0.0,43604.0,10940018.0,0.0,33426.0,0.0,0.0,0.0,0.0,...,25529.0,0.0,24866.0,180691.0,8200.0,0.0,0.0,833.0,380000.0,0.0
2009-01-22,0.0,0.0,22565.0,8395487.0,0.0,92773.0,0.0,0.0,0.0,194355.0,...,64279.0,0.0,7525.0,1300.0,23000.0,0.0,244.0,5000.0,35000.0,0.0
